# バイナリデータの取り扱い - 1
参考：  
* https://izadori.net/python-binaryfile/
* https://tabinou.com/archives/1511
* https://atmarkit.itmedia.co.jp/ait/articles/2105/18/news019.html
* https://www.tutimogura.com/python-binaryfile-read/
* https://qiita.com/katsuki104/items/3d0fbcb5c7da19d318bd
* [Pythonのstructモジュール公式ドキュメント](https://docs.python.org/ja/3/library/struct.html)

In [197]:
import struct
import os
import time
import datetime
import numpy as np

## バイナリで書き込まれた整数を読み取り、テキスト形式で書きだす

In [1]:
# バイナリデータの書き込み
f = open('input/myfile.dat',mode="wb")
for i in range(32):
    f.write(i.to_bytes(1,"big"))
f.close()

In [2]:
#バイナリデータをテキストデータに変換して書き込み
with open('input/myfile.dat', 'rb') as f:
    binary_data = f.read()

# 1バイトごとにデータを読み取る
integers = [int.from_bytes(binary_data[i:i+1], byteorder='big') for i in range(0, len(binary_data), 1)]

# テキストファイルに書き出す
with open('output.txt', 'w') as f:
    for integer in integers:
        f.write(f"{integer}\n")


In [3]:
binary_data

b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f'

## バイナリデータとテキストデータが混ざったファイルの取り扱い - 1
目的：　バイナリデータとテキストデータが混ざっているファイルのバイナリデータ部分をテキストに変換する

書き込んだ結果はバイナリエディタで確認するとよい。  
上記で作成したファイルは先頭6バイトがテキストで、残り40バイトがバイナリとなっているレコードが10行あるファイルとなる。  
バイト列を10進数などに変換する際はバイトオーダーに注意。  
https://corgi-lab.com/programming/c-lang/warn-endian/


In [39]:
# 10レコードのデータを作成する
num_records = 10

# ファイル名を指定する
filename = 'input/binary_and_text.dat'

# ファイルをバイナリモードで開く
with open(filename, 'wb') as file:
    for i in range(num_records):
        # テキストデータの作成。:02dは2桁でゼロパディング
        text_data = f"Text{i:02d}".encode('utf-8')  # 6バイトのテキストデータを作成

        # バイナリデータの作成。iはCの型intを表し、4Byteの整数型
        binary_data = struct.pack('10i', i, i+1, i+2, i+3, i+4, i+5, i+6, i+7, i+8, i+9)  # 40バイトのバイナリデータを作成

        # テキストデータとバイナリデータをファイルに書き込む
        file.write(text_data)
        file.write(binary_data)

# ファイルサイズを確認する
file_size = os.path.getsize(filename)
print(f'File size: {file_size} bytes')  # 出力: File size: 46*100 bytes


File size: 460 bytes


In [63]:
txt_data = []
bin_data = []
# 取り出したいバイナリ部分の開始バイト
target_start_offset = 6
# １レコードあたりのバイト数
record_length = 46

with open(filename, 'rb') as file:
    contents = file.read()
    for i in range(num_records):
        # テキスト部分の取り出し
        temp = struct.unpack_from('6c', contents, offset= record_length * i)
        txt_data.append(temp)
        # バイナリ部分の取り出し
        temp = struct.unpack_from('10i', contents, offset= target_start_offset + record_length * i)
        bin_data.append(temp)

In [64]:
txt_data

[(b'T', b'e', b'x', b't', b'0', b'0'),
 (b'T', b'e', b'x', b't', b'0', b'1'),
 (b'T', b'e', b'x', b't', b'0', b'2'),
 (b'T', b'e', b'x', b't', b'0', b'3'),
 (b'T', b'e', b'x', b't', b'0', b'4'),
 (b'T', b'e', b'x', b't', b'0', b'5'),
 (b'T', b'e', b'x', b't', b'0', b'6'),
 (b'T', b'e', b'x', b't', b'0', b'7'),
 (b'T', b'e', b'x', b't', b'0', b'8'),
 (b'T', b'e', b'x', b't', b'0', b'9')]

バイト型になっているのでstrにしたい場合はUtf-8でエンコーディングすればよい。

In [82]:
converted_txt_data = []

for tup in txt_data:
    temp = b''
    for i in range(len(tup)):
        # unpack結果を結合
        temp += tup[i]

    # 結合したバイト型をUtf-8にデコード
    converted_txt_data.append(temp.decode('Utf-8'))

In [81]:
converted_txt_data

['Text00',
 'Text01',
 'Text02',
 'Text03',
 'Text04',
 'Text05',
 'Text06',
 'Text07',
 'Text08',
 'Text09']

In [56]:
bin_data

[(0, 1, 2, 3, 4, 5, 6, 7, 8, 9),
 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10),
 (2, 3, 4, 5, 6, 7, 8, 9, 10, 11),
 (3, 4, 5, 6, 7, 8, 9, 10, 11, 12),
 (4, 5, 6, 7, 8, 9, 10, 11, 12, 13),
 (5, 6, 7, 8, 9, 10, 11, 12, 13, 14),
 (6, 7, 8, 9, 10, 11, 12, 13, 14, 15),
 (7, 8, 9, 10, 11, 12, 13, 14, 15, 16),
 (8, 9, 10, 11, 12, 13, 14, 15, 16, 17),
 (9, 10, 11, 12, 13, 14, 15, 16, 17, 18)]

In [54]:
# def write_bytes(filename, start_byte, end_byte, data):
#     # 範囲のバイト数を計算
#     byte_range = end_byte - start_byte + 1

#     # 書き込むデータの長さが範囲のバイト数と一致することを確認
#     if len(data) != byte_range:
#         raise ValueError(f'Data length does not match byte range: {len(data)} != {byte_range}')

#     # ファイルをバイナリ書き込みモードで開く
#     with open(filename, 'r+b') as file:
#         # Xバイト目にファイルポインタを移動
#         file.seek(start_byte)

#         # データを書き込む
#         file.write(data)

# # 使用例
# filename = 'mixed_data.dat'  # 対象のファイル名
# start_byte = 10  # 開始バイト位置 (0インデックス)
# end_byte = 19  # 終了バイト位置 (0インデックス)
# data = b'new binary'  # 書き込むバイナリデータ

# write_bytes(filename, start_byte, end_byte, data)

## バイナリデータとテキストデータが混ざったファイルの取り扱い - 2
目的：　より複雑なバイナリデータ・テキストデータ混在のファイルについて取り扱う

In [119]:
# 10レコードのデータを作成する
num_records = 10

# ファイル名を指定する
filename = 'input/binary_and_text_3.dat'

# ファイルをバイナリモードで開く
with open(filename, 'wb') as file:
    for i in range(num_records):
        # テキストデータの作成。:02dは2桁でゼロパディング
        text_data_1 = f"Text{i:02d}".encode('utf-8')  # 6バイトのテキストデータを作成

        # バイナリデータの作成。iはCの型intを表し、4Byteの整数型
        binary_data_1 = struct.pack('10i', i, i+1, i+2, i+3, i+4, i+5, i+6, i+7, i+8, i+9)  # 40バイトのバイナリデータを作成

        # テキストデータの作成。:02dは2桁でゼロパディング
        text_data_2 = f"AAAA{i:03d}".encode('utf-8')  # 7バイトのテキストデータを作成

        # エポック秒の秒以上をlong longで作成
        epoch = time.time()
        print(f'エポック秒{epoch}')
        epoch_s = str(epoch).split('.')[0]
        binary_data_2 = struct.pack('q', int(epoch_s))  # 8バイト

        # エポック秒のミリ秒以下をlongで作成
        epoch_msusns = str(epoch).split('.')[1]
        binary_data_3 = struct.pack('l', int(epoch_msusns))  # 8バイト。ドキュメントでは4バイトと書かれているが、、

        # テキストデータの作成。
        text_data_3 = "END".encode('utf-8')  # 3バイトのテキストデータを作成


        # テキストデータとバイナリデータをファイルに書き込む
        file.write(text_data_1)
        file.write(binary_data_1)
        file.write(text_data_2)
        file.write(binary_data_2)
        file.write(binary_data_3)
        file.write(text_data_3)


# ファイルサイズを確認する
file_size = os.path.getsize(filename)
print(f'File size: {file_size} bytes')  # 出力: File size: 72*10 bytes


エポック秒1695518901.3639977
エポック秒1695518901.3644247
エポック秒1695518901.3645647
エポック秒1695518901.3646924
エポック秒1695518901.3648338
エポック秒1695518901.3649383
エポック秒1695518901.366534
エポック秒1695518901.3666852
エポック秒1695518901.3668692
エポック秒1695518901.367029
File size: 720 bytes


出力結果のファイルをバイナリエディタで見ると下記のようになる。  
![Alt text](image-1.png)

In [136]:
# ファイルにちゃんとエポック秒が書き込まれているか確認。リトルエンディアンなので注意。
int('00000000650F90B5', 16)

1695518901

### 作成したファイルからエポック秒、エポック秒（ミリ秒以下）だけ取り出す
やることは同じ。欲しい情報がバイナリデータのどの位置にあり、どのような型（バイト長）で定義されているのか知っていないと、  
適切に取り出すことは出来ない。

In [189]:
dict_epoch = {}
filename = 'input/binary_and_text_3.dat'

# １レコードあたりのバイト数
record_length = 72

# 各項目のオフセットとデータ型を定義。
# 欲しい情報がバイナリデータのどの位置にあり、どのような型（バイト長）で定義されているのか知らないと取り出せない。
block_def = {
    'epoch_s'         : {'offset': 53, 'type': '<q'},
    'epoch_msusns'         : {'offset': 61, 'type': '<l'},
}

# 結果格納用の辞書にキー：空リストを作っておく
for k in block_def.keys(): 
    dict_epoch[k] = [ ] 


with open(filename, 'rb') as file:
    contents = file.read()
    for i in range(num_records):
        for k, v in block_def.items():
            # 欲しい箇所の取り出し
            temp = struct.unpack_from(f'{v["type"]}', contents, offset= v["offset"] + record_length * i)
            dict_epoch[k].append(str(temp[0]))
        # # バイナリ部分の取り出し
        # temp = struct.unpack_from('10i', contents, offset= target_start_offset + record_length * i)
        # bin_data.append(temp)

In [190]:
dict_epoch

{'epoch_s': ['1695518901',
  '1695518901',
  '1695518901',
  '1695518901',
  '1695518901',
  '1695518901',
  '1695518901',
  '1695518901',
  '1695518901',
  '1695518901'],
 'epoch_msusns': ['3639977',
  '3644247',
  '3645647',
  '3646924',
  '3648338',
  '3649383',
  '366534',
  '3666852',
  '3668692',
  '367029']}

In [194]:
ziped_epoch = list(zip(dict_epoch['epoch_s'], dict_epoch['epoch_msusns']))
result = [float(v1 + '.' + v2) for v1, v2 in ziped_epoch]

In [196]:
result

[1695518901.3639977,
 1695518901.3644247,
 1695518901.3645647,
 1695518901.3646924,
 1695518901.3648338,
 1695518901.3649383,
 1695518901.366534,
 1695518901.3666852,
 1695518901.3668692,
 1695518901.367029]

In [200]:
# タイムゾーンの生成
JST = datetime.timezone(datetime.timedelta(hours=+9), 'JST')

# 取得したエポック秒を時刻に変換
datetime.datetime.fromtimestamp(result[0], tz=JST)

datetime.datetime(2023, 9, 24, 10, 28, 21, 363998, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), 'JST'))